Linear Regression model to predict admission 

In [25]:
import findspark
findspark.init()

In [26]:
#create a sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('spark').getOrCreate()

In [27]:
#create a spark dataframe

df=spark.read.csv('Admission_Predict_Ver1.1.csv',header=True,inferSchema=True)

In [28]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [29]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [30]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [33]:
#get the summary statistics
df.describe().show(vertical=True)

-RECORD 0--------------------------------
 summary           | count               
 Serial No         | 500                 
 GRE Score         | 500                 
 TOEFL Score       | 500                 
 University Rating | 500                 
 SOP               | 500                 
 LOR               | 500                 
 CGPA              | 500                 
 Research          | 500                 
 Chance of Admit   | 500                 
-RECORD 1--------------------------------
 summary           | mean                
 Serial No         | 250.5               
 GRE Score         | 316.472             
 TOEFL Score       | 107.192             
 University Rating | 3.114               
 SOP               | 3.374               
 LOR               | 3.484               
 CGPA              | 8.576440000000003   
 Research          | 0.56                
 Chance of Admit   | 0.7217399999999996  
-RECORD 2--------------------------------
 summary           | stddev       

# TASK 3 : Data Cleaning

In [34]:
#drop the unnecessary column
df = df.drop ('Serial No')

In [35]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [36]:
#check for null values
for i in df.columns:
  print(i+":",df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [37]:
# correlation analysis
for col in df.columns:
  print("Correlation to chance of admit col for {} is {}".format(col,df.stat.corr('chance of Admit',col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [38]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA'],outputCol='features')
output_data =assembler.transform(df)

In [39]:
#display dataframe
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [40]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features', 'chance of Admit')

In [41]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- chance of Admit: double (nullable = true)



In [42]:
#split the dataset into training and testing set
train,test =final_data.randomSplit([0.75,0.25])

In [43]:
#build & train the model
models = LinearRegression(featuresCol = 'features', labelCol ='chance of Admit')
model =models.fit(train)

In [44]:
#get coefficients & intercept
print("COEFICIENTS:", model.coefficients)
print('intercept:',model.intercept)


COEFICIENTS: [0.0024773462111568573,0.00354081743532739,0.1401319232819739]
intercept: -1.6448938150301282


In [45]:
#get summary of the model
summary = model.summary
print(summary)

In [46]:
#print the rmse & r2 score
print('RMSE', summary.rootMeanSquaredError)
print('r2 score', summary.r2)
#print()

RMSE 0.06228334315469713
r2 score 0.8033680701382455


### TASK 6 : Evaluate the model

In [47]:
#transform on the test data
predictions = model.transform(test)

In [48]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|chance of Admit|         prediction|
+------------------+---------------+-------------------+
|  [293.0,97.0,7.8]|           0.64| 0.5174569176649844|
|  [295.0,93.0,7.2]|           0.46|  0.424169186376804|
| [295.0,99.0,7.65]|           0.57| 0.5084734564656566|
| [296.0,99.0,7.28]|           0.47| 0.4591019910624834|
| [297.0,96.0,7.89]|           0.43|  0.536437358169662|
| [298.0,98.0,8.03]|           0.34|   0.56561480851095|
|  [298.0,99.0,7.6]|           0.46| 0.5088988989350283|
|[298.0,101.0,7.69]|           0.53| 0.5285924069010612|
|[298.0,105.0,8.54]|           0.69|  0.661867811432048|
|[299.0,100.0,8.02]|           0.63| 0.5737724703599418|
| [300.0,95.0,8.22]|           0.62| 0.5865721140508564|
|  [300.0,99.0,6.8]|           0.36|0.40174805273176317|
|[300.0,102.0,8.17]|           0.63| 0.6043512399340492|
| [301.0,99.0,8.22]|           0.64|  0.603212730003323|
|[301.0,102.0,8.13]|           

In [49]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='chance of Admit', metricName ='r2')
print('r2 on the test data', evaluator.evaluate(predictions)) 

r2 on the test data 0.8010802427208986


#### metricName: 

1. rmse - root mean squared error (default)\n

2. mse - mean squared error

3. r2 - r^2 metric

4. mae - mean absolute error

5. var - explained variance.')
